In [2]:
# Importamos las librerías estándar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import math

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# Descargamos los datos
url = 'https://drive.google.com/file/d/1lqdL5E3pjC0ISjsEe_F1N8xIKK8y-0nz/view?usp=sharing'
file_id = url.split('/')[-2]
download_url = 'https://drive.google.com/uc?id=' + file_id

df = pd.read_csv(download_url)

In [ ]:
# Link al repo con información adicional
# https://github.com/santy-gegenschatz/CoderHouse-25570-Proyecto-Spotify

In [ ]:
# Limpiamos los datos
df = df.set_index('id')
df = df.dropna()
df = df.drop_duplicates()
df.shape

In [ ]:
# Describimos los datos
df.describe(include = 'all').T

In [ ]:
# Antes de hacer el análisis, vamos a hacer una variable que promedie cada una de las 3 variables diferentes por álbum
df['average_duration'] = ( df['t_dur0'] + df['t_dur1'] + df['t_dur2'] ) / 3
df['average_dance'] = ( df['t_dance0'] + df['t_dance1'] + df['t_dance2'] ) / 3
df['average_energy'] = ( df['t_energy0'] + df['t_energy1'] + df['t_energy2'] ) / 3
df['average_mode'] = ( df['t_mode0'] + df['t_mode1'] + df['t_mode2'] ) / 3
df['average_speech'] = ( df['t_speech0'] + df['t_speech1'] + df['t_speech2'] ) / 3
df['average_acous'] = ( df['t_acous0'] + df['t_acous1'] + df['t_acous2'] ) / 3
df['average_ins'] = ( df['t_ins0'] + df['t_ins1'] + df['t_ins2'] ) / 3
df['average_live'] = ( df['t_live0'] + df['t_live1'] + df['t_live2'] ) / 3
df['average_val'] = ( df['t_val0'] + df['t_val1'] + df['t_val2'] ) / 3
df['average_tempo'] = ( df['t_tempo0'] + df['t_tempo1'] + df['t_tempo2'] ) / 3
df['average_sig'] = ( df['t_sig0'] + df['t_sig1'] + df['t_sig2'] ) / 3

In [ ]:
df.describe().T

In [ ]:
# Ahora hacemos un nuevo dataframe con las columnas promediadas y la columna de salida (popularity)
df2 = df[['average_duration', 'average_dance', 'average_energy', 'average_mode', 'average_speech', 'average_acous', 'average_live', 'average_val', 'average_tempo', 'average_sig', 'popularity']].copy()

In [ ]:
df2.describe().T

In [ ]:
df2.head()

In [ ]:
# Ahora podemos entrenar un modelo en función de todas las variables numéricas que prediga la popularidad

In [ ]:
# Para el análisis multivariado, vamos a realizar 3 gráficos.
# Uno es un pairplot, otro es un gráifco radial y otro es un gráfico en 3d.

## Pairplot

In [29]:
# Para el pairplot vamos a tomar la categoría popularity y la vamos a partir en tres, y vamos a graficar el pairplot con un hue de cada una de esas 3 categorías

In [21]:
def categorize(value):
    if (value <= 33):
        return "LOW"
    elif (value <= 67):
        return "MEDIUM"
    else:
        return "HIGH"

In [22]:
df2['type_of_popularity'] = df2['popularity'].apply(lambda value: categorize(value))

In [23]:
df2.head()

,average_duration,average_dance,average_energy,average_mode,average_speech,average_acous,average_live,average_val,average_tempo,average_sig,popularity,type_of_popularity
id,,,,,,,,,,,,
0,192284.000000,0.536667,0.458667,0.666667,0.048967,0.490333,0.091233,0.779000,133.243333,3.333333,51,MEDIUM
1,195199.666667,0.779000,0.658333,0.000000,0.202100,0.109367,0.114333,0.413333,139.028000,4.000000,67,MEDIUM
2,154768.333333,0.555000,0.487000,0.666667,0.272667,0.240847,0.286733,0.521333,122.268667,4.000000,81,HIGH
3,203866.333333,0.628333,0.585000,1.000000,0.027467,0.333333,0.203333,0.556333,114.783333,4.000000,53,MEDIUM
4,140693.000000,0.638000,0.302333,1.000000,0.071500,0.711333,0.270667,0.533000,116.460000,4.000000,70,HIGH
